Installations

In [4]:
!pip install neo4j-driver
!pip install gradio


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\kiahm\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


  Using cached typer-0.12.3-py3-none-any.whl.metadata (15 kB)
Using cached typer-0.12.3-py3-none-any.whl (47 kB)
  Attempting uninstall: typer
    Found existing installation: typer 0.9.4
    Uninstalling typer-0.9.4:
      Successfully uninstalled typer-0.9.4


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spacy 3.7.4 requires typer<0.10.0,>=0.3.0, but you have typer 0.12.3 which is incompatible.
weasel 0.3.4 requires typer<0.10.0,>=0.3.0, but you have typer 0.12.3 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\kiahm\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --------------------------------------- 0.0/12.8 MB 991.0 kB/s eta 0:00:13
     ---------------------------------------- 0.1/12.8 MB 1.3 MB/s eta 0:00:10
     ---------------------------------------- 0.1/12.8 MB 1.2 MB/s eta 0:00:11
      --------------------------------------- 0.2/12.8 MB 1.1 MB/s eta 0:00:12
      --------------------------------------- 0.3/12.8 MB 1.2 MB/s eta 0:00:11
      --------------------------------------- 0.3/12.8 MB 1.2 MB/s eta 0:00:11
     - -------------------------------------- 0.4/12.8 MB 1.1 MB/s eta 0:00:11
     - -------------------------------------- 0.4/12.8 MB 1.2 MB/s eta 0:00:11
     - -------------------------------------- 0.5/12.8 MB 1.2 MB/s eta 0:00:11
     - -------------------------------------- 0.5/12.8 MB 1.2 MB/s eta 0:00:11
     - -------------------------------------- 0.6/12.8 MB 1.2 MB/s eta 0:00:11
     - -------------------------------------- 0.6/12.8 MB 

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 4.40.0 requires typer<1.0,>=0.12; sys_platform != "emscripten", but you have typer 0.9.4 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\kiahm\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Imports

In [2]:
from neo4j import GraphDatabase
import spacy
import gradio as gr

C:\Users\kiahm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
driver = GraphDatabase.driver("bolt://44.222.165.210:7687",
                              auth = ("neo4j", "servants-overtime-rainbows"))

In [4]:
nlp = spacy.load("en_core_web_sm")

Functions to obtain released dates

In [5]:
def extract_entity(qn, entity_type):
    doc = nlp(qn)
    for ent in doc.ents:
        if ent.label_ in entity_type:
            return ent.text
    return None

In [6]:
def chatbot(input, hist = []):
    output = get_ans(input)
    hist.append((input, output))
    return hist, hist

In [7]:
def get_ans(qn):
    doc = nlp(qn)
    title = extract_entity(qn, ["WORK_OF_ART", "MOVIE"])
    director = extract_entity(qn, ["PERSON"])
    
    if title is None:
        return "Sorry, I don't know that movie."
    
    with driver.session() as session:
        # Release date query
        if "release" in qn or "released" in qn:
            result = session.run(f"MATCH(m:Movie) WHERE m.title = '{title}' RETURN m.released") #Get release date
            if result.peek() is None:
                return f"No infomation found for movie '{title}'"
            for record in result:
                return f"'{title}' was released in {record['m.released']}"
            
        # Tagline query
        elif "tagline" in qn:
            result = session.run(f"MATCH(m:Movie) WHERE m.title = '{title}' RETURN m.tagline")
            if result.peek() is None:
                return f"No information found for tagline of movie '{title}'"
            for record in result:
                return f"The tagline of '{title}' is {record['m.tagline']}."
            
        # Director query
        elif "director" in qn or "directed" in qn:
            query = f"MATCH (m:Movie {{title: '{title}'}})<-[d:DIRECTED]-(p:Person) RETURN p.name"
            print(f"Running query: {query}")  # Debug: Print the query
            result = session.run(query)
            print(result)
            if result.peek() is None:
                return f"No information found for director of movie '{title}'"
            for record in result:
                return f"'{title}' was directed by {record['p.name']}."
        else:
            return "Sorry, I don't understand your question."

Chatbot Frontend

In [8]:
gr.Interface(fn=chatbot,
             inputs=["text", "state"],
             outputs=["chatbot", "state"]).launch(debug=True)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Running query: MATCH (m:Movie {title: 'RescueDawn'})<-[d:DIRECTED]-(p:Person) RETURN p.name
Keyboard interruption in main thread... closing server.
